In [1]:
# from image_processing import *
import dlib
import cv2
import numpy as np

# Load the detector
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [2]:
def extract_eye_region(image, landmarks, eye_points, buffer= 4):
    # Extract the coordinates of the eye points
    region = np.array([(landmarks.part(point).x, landmarks.part(point).y) for point in eye_points])
    # Create a mask with zeros
    height, width = image.shape[:2]
    mask = np.zeros((height, width), np.uint8)
    # Fill the mask with the polygon defined by the eye points
    cv2.fillPoly(mask, [region], 255)
    # Bitwise AND operation to isolate the eye region
    eye = cv2.bitwise_and(image, image, mask=mask)
    # Cropping the eye region
    (min_x, min_y) = np.min(region, axis=0)
    (max_x, max_y) = np.max(region, axis=0)
    min_x = max(min_x - buffer, 0)
    min_y = max(min_y - buffer, 0)
    max_x = min(max_x + buffer, width)
    max_y = min(max_y + buffer, height)
    cropped_eye = eye[min_y:max_y, min_x:max_x]
    return cropped_eye, (min_x, min_y, max_x, max_y)


In [3]:
import cv2

def enhance_image_resolution(image):
    # Load the super-resolution model
    sr = cv2.dnn_superres.DnnSuperResImpl_create()
    path = "EDSR_x4.pb"  # Change to the path of the model
    sr.readModel(path)
    sr.setModel("edsr", 4)  # You can change the model and scale as needed

    # Enhance the resolution of the image
    enhanced_image = sr.upsample(image)
    return enhanced_image


In [4]:
def is_pupil_candidate(contour, gray_eye_image, eye_center, threshold=0.8):
    # Calculate area and perimeter of the contour
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)

    # Check for circularity
    circularity = 4 * np.pi * (area / (perimeter * perimeter))
    if circularity < threshold:
        return False

    # Check aspect ratio of bounding rectangle
    x, y, w, h = cv2.boundingRect(contour)
    aspect_ratio = float(w) / h
    if aspect_ratio < 0.8 or aspect_ratio > 1.2:
        return False

    # Check solidity
    hull = cv2.convexHull(contour)
    hull_area = cv2.contourArea(hull)
    solidity = float(area) / hull_area
    if solidity < threshold:
        return False

    # Check relative location to the center of the eye
    M = cv2.moments(contour)
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    distance_to_center = np.sqrt((cX - eye_center[0])**2 + (cY - eye_center[1])**2)
    if distance_to_center > (w / 2):
        return False

    # Check intensity
    mask = np.zeros(gray_eye_image.shape, np.uint8)
    cv2.drawContours(mask, [contour], -1, 255, -1)
    mean_val = cv2.mean(gray_eye_image, mask=mask)[0]
    if mean_val > 50:  # This threshold can be adjusted based on your images
        return False

    return True

In [5]:
def detect_pupil(eye_image, eye_center):
    # Enhance resolution
    eye_image = enhance_image_resolution(eye_image)

    # Preprocessing
    gray = cv2.cvtColor(eye_image, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)  # Histogram Equalization
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)  # Gaussian Blur

    # Edge detection
    edged = cv2.Canny(blurred, 50, 100)

    # Find contours
    contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort contours by area and filter
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    # Assuming the eye center is passed as a parameter, you can use it directly.
    # If you need to calculate it, you can use eye landmarks like this:
    # eye_center = np.mean(eye_landmarks, axis=0) where eye_landmarks is an array of (x, y) tuples

    for contour in contours:
        if is_pupil_candidate(contour, gray, eye_center):
            # Found a good pupil candidate
            M = cv2.moments(contour)
            if M["m00"] != 0:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                cv2.circle(eye_image, (cX, cY), 7, (255, 255, 255), -1)
                cv2.imshow("Pupil", eye_image)
                cv2.waitKey(0)
                cv2.destroyAllWindows()
                return (cX, cY), contour

    return None, None


In [6]:
def pre_process_image(image):        
    # Initialize variables
    left_eye_info = right_eye_info = left_eye_bbox = right_eye_bbox = None

    dlib_faces = detector(image)
    processed_data = []
    for dlib_face in dlib_faces:
        shape = predictor(image, dlib_face)

        for (i, (start, end)) in enumerate([(36,42), (42,48)]):
            eye_landmarks = [(shape.part(point).x, shape.part(point).y) for point in range(start, end)]
            eye_center = np.mean(eye_landmarks, axis=0).astype(int)  # Ensure you have integers for the center
            eye_image, (eye_min_x, eye_min_y, eye_max_x, eye_max_y) = extract_eye_region(image, shape, range(start, end))
            print(eye_max_x, eye_max_y)
            print(eye_min_x, eye_min_y)
# Now call the detect_pupil function with the eye image and the calculated eye center
            pupil_center, pupil_contour = detect_pupil(eye_image, eye_center)
            # After detecting the pupil in the cropped eye image:
            if pupil_center:
                # Scale the pupil center coordinates down to the original image size
                pupil_center_original = (pupil_center[0] / 4, pupil_center[1] / 4)
                # Transform these coordinates to the global space of the original image
                pupil_center_global = (int(pupil_center_original[0]) + eye_min_x, int(pupil_center_original[1]) + eye_min_y)
    
                pupil_center_global = tuple(pc.item() if isinstance(pc, np.generic) else pc for pc in pupil_center_global)

                #draw contours
                cv2.drawContours(image, [pupil_contour], -1, (0, 255, 0), 2)
                cv2.circle(image, pupil_center_global, 1, (255, 255, 255), -1)
                cv2.imshow("Eye", image)
                cv2.waitKey(0)
                cv2.destroyAllWindows()


                bounding_box = (eye_min_x, eye_min_y, eye_max_x - eye_min_x, eye_max_y - eye_min_y)
                bounding_box = tuple(bb.item() if isinstance(bb, np.generic) else bb for bb in bounding_box)

                eye_data = {
                    'eye_position': 'left' if i == 0 else 'right',
                    'pupil_center': pupil_center_global,
                    'bounding_box': bounding_box
                }
                processed_data.append(eye_data)

        # Processed data for each eye
    for eye_data in processed_data:
        if eye_data['eye_position'] == 'left':
            left_eye_info = eye_data['pupil_center']
            left_eye_bbox = eye_data['bounding_box']
        else:
            right_eye_info = eye_data['pupil_center']
            right_eye_bbox = eye_data['bounding_box']
    
    # Check if any eye information was detected
    if left_eye_info is None and right_eye_info is None:
        print("No eye information detected")
        return None

    return processed_data, left_eye_info, right_eye_info, left_eye_bbox, right_eye_bbox, shape
    

In [7]:
image = 'data/Naia/calibration_images/Naia_07779f08-1c1f-490f-9d26-7786c43aca1d.png'
pre_process_image(cv2.imread(image))
# # Load your image here
# image = cv2.imread(image)
# eye_image = extract_eye_region(image)


333 175
289 157


ZeroDivisionError: float division by zero

In [18]:
image1 = 'data/Will/eye_gaze_images/Will_01bdf4b4-38da-4bd3-a9b4-0ff7b57bd23a.png'
pre_process_image(cv2.imread(image1))

308 272
269 252
4364.0
383 271
346 252
81.5


([{'eye_position': 'left',
   'pupil_center': (289, 262),
   'bounding_box': (269, 252, 39, 20)},
  {'eye_position': 'right',
   'pupil_center': (365, 261),
   'bounding_box': (346, 252, 37, 19)}],
 (289, 262),
 (365, 261),
 (269, 252, 39, 20),
 (346, 252, 37, 19),
 <_dlib_pybind11.full_object_detection at 0x2b20dc2f4b0>)